In [ ]:
!pip install -q ogb networkx rank-bm25 sentence-transformers pypdf python-docx

import pandas as pd
import networkx as nx
import torch
import gzip
import os
import io
import requests
import urllib.parse
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
from google.colab import files
from pypdf import PdfReader
from IPython.display import display, Markdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.1 MB/s eta 0:00:00


In [ ]:
# --- A. PYTORCH 2.6 FIX (Your Patch) ---
_original_torch_load = torch.load
def permissive_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _original_torch_load(*args, **kwargs)
torch.load = permissive_load

In [ ]:
from ogb.nodeproppred import NodePropPredDataset
dataset = NodePropPredDataset(name='ogbn-arxiv', root='dataset')

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:12<00:00,  6.30it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 12264.05it/s]

Saving...


In [ ]:
graph = dataset.graph

In [ ]:
graph

{'edge_index': array([[104447,  15858, 107156, ...,  45118,  45118,  45118],
        [ 13091,  47283,  69161, ..., 162473, 162537,  72717]]),
 'edge_feat': None,
 'node_feat': array([[-0.057943, -0.05253 , -0.072603, ...,  0.173364, -0.172796,
         -0.140059],
        [-0.1245  , -0.070665, -0.325202, ...,  0.068524, -0.372111,
         -0.301036],
        [-0.080242, -0.023328, -0.183787, ...,  0.109919,  0.117589,
         -0.139883],
        ...,
        [-0.22053 , -0.036568, -0.402199, ...,  0.11336 , -0.161393,
         -0.145171],
        [-0.138236,  0.040885, -0.251811, ..., -0.08929 , -0.041253,
         -0.376132],
        [-0.029875,  0.268417, -0.161124, ...,  0.120807,  0.077647,
         -0.091018]], dtype=float32),
 'node_year': array([[2013],
        [2015],
        [2014],
        ...,
        [2020],
        [2020],
        [2020]]),
 'num_nodes': 169343}

In [ ]:
text_url = "https://snap.stanford.edu/ogb/data/misc/ogbn_arxiv/titleabs.tsv.gz"
text_file = "titleabs.tsv.gz"
if not os.path.exists(text_file):
    r = requests.get(text_url)
    with open(text_file, 'wb') as f: f.write(r.content)

import gzip
import shutil

with gzip.open('titleabs.tsv.gz', 'rb') as f_in:
    with open('titleabs.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)




In [ ]:
import pandas as pd
df = pd.read_csv('titleabs.tsv', sep='\t', names=['paper_id', 'title', 'abstract'])


In [ ]:
import os
import pickle
import torch
import pandas as pd
import networkx as nx
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from ogb.nodeproppred import NodePropPredDataset # Uncomment if needed

In [ ]:
# 1. SETUP DATA (Same as before)
dataset = NodePropPredDataset(name='ogbn-arxiv', root='dataset')
graph = dataset.graph
text_url = "https://snap.stanford.edu/ogb/data/misc/ogbn_arxiv/titleabs.tsv.gz"
text_file = "titleabs.tsv.gz"
if not os.path.exists(text_file):
    r = requests.get(text_url)
    with open(text_file, 'wb') as f: f.write(r.content)

import gzip
import shutil

with gzip.open('titleabs.tsv.gz', 'rb') as f_in:
    with open('titleabs.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

df = pd.read_csv('titleabs.tsv', sep='\t', names=['paper_id', 'title', 'abstract'])
df['text'] = df['title'].fillna('') + " " + df['abstract'].fillna('')


In [ ]:
# 2. CREATE SAVE DIRECTORY
os.makedirs("search_data", exist_ok=True)# 2. CREATE SAVE DIRECTORY

In [ ]:
# 3. BUILD & SAVE GRAPH
print("Building and saving Graph...")
G = nx.DiGraph()
src, dst = graph['edge_index']
G.add_edges_from(zip(src, dst))
with open("search_data/graph.pkl", "wb") as f:
    pickle.dump(G, f)

Building and saving Graph...


In [ ]:
# 4. BUILD & SAVE BM25 INDEX
print("Building and saving BM25 Index...")
tokenized_corpus = [doc.lower().split() for doc in df['text']]
bm25 = BM25Okapi(tokenized_corpus)
with open("search_data/bm25.pkl", "wb") as f:
    pickle.dump(bm25, f)

Building and saving BM25 Index...


In [ ]:
# 5. COMPUTE & SAVE BERT EMBEDDINGS
print("Encoding all papers (this takes time)...")
model = SentenceTransformer('all-MiniLM-L6-v2')
# Encode ALL papers (remove the slice [:10000] for production)
corpus_embeddings = model.encode(df['text'].tolist(), convert_to_tensor=True, show_progress_bar=True)
torch.save(corpus_embeddings, "search_data/embeddings.pt")
# 6. SAVE DATAFRAME
print("Saving DataFrame...")
df.to_pickle("search_data/df.pkl")

Encoding all papers (this takes time)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5617 [00:00<?, ?it/s]

Saving DataFrame...


In [ ]:
import pickle
import torch
import pandas as pd
import networkx as nx
import numpy as np
import urllib.parse
from sentence_transformers import SentenceTransformer, util

class PaperSearchEngine:
    def __init__(self, data_dir="search_data"):
        print("Loading search engine components...")
        self.df = pd.read_pickle(f"{data_dir}/df.pkl")

        with open(f"{data_dir}/graph.pkl", "rb") as f:
            self.G = pickle.load(f)

        with open(f"{data_dir}/bm25.pkl", "rb") as f:
            self.bm25 = pickle.load(f)

        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.corpus_embeddings = torch.load(f"{data_dir}/embeddings.pt", weights_only=True)
        print("System Ready!\n")

    def _generate_link(self, title):
        """
        Generates a direct arXiv search link since OGB does not provide arXiv IDs.
        We quote the title to ensure an exact match.
        """
        # Encode the title for URL (e.g., spaces -> %20)
        safe_title = urllib.parse.quote(f'"{title}"')
        return f"https://arxiv.org/search/?query={safe_title}&searchtype=title"

    # --- INTERNAL RETRIEVAL LOGIC ---

    def _get_bm25(self, query, top_k=100):
        tokenized_query = query.lower().split()
        scores = self.bm25.get_scores(tokenized_query)
        top_n = np.argsort(scores)[::-1][:top_k]
        return [(idx, scores[idx]) for idx in top_n if scores[idx] > 0]

    def _get_bert(self, query, top_k=100):
        query_embedding = self.model.encode(query, convert_to_tensor=True)
        cos_scores = util.cos_sim(query_embedding, self.corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)
        return [(idx.item(), score.item()) for idx, score in zip(top_results.indices, top_results.values)]

    def _get_pagerank(self, query, top_k=100):
        # 1. Get Seeds from BM25
        bm25_results = self._get_bm25(query, top_k=100)

        # 2. Build Personalization Dict
        personalization = {idx: score for idx, score in bm25_results}

        if not personalization:
            return []

        # 3. Run PageRank
        pr_scores = nx.pagerank(self.G, alpha=0.85, personalization=personalization)

        # 4. Sort and return
        return sorted(pr_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

    # --- PUBLIC FACING METHODS ---

    def print_results(self, header, results, top_k=5):
        print(f"\n--- {header} ---")
        if not results:
            print("No results found.")
            return

        for rank, (idx, score) in enumerate(results[:top_k]):
            if idx >= len(self.df): continue
            paper = self.df.iloc[idx]

            # Print cleanly
            print(f"{rank+1}. [Score: {score:.4f}] {paper['title']}")
            print(f"   Link: {self._generate_link(paper['title'])}")
            print(f"   Abstract: {paper['abstract'][:100]}...")
            print("")

    def compare_methods(self, query):
        print(f"\n{'='*20} COMPARING METHODS: {query} {'='*20}")
        self.print_results("BM25 (Keywords)", self._get_bm25(query))
        self.print_results("BERT (Semantic)", self._get_bert(query))
        self.print_results("PageRank (Authority)", self._get_pagerank(query))

# --- RUN IT ---
engine = PaperSearchEngine()
engine.compare_methods("graph neural networks")

Loading search engine components...
System Ready!


==================== COMPARING METHODS: graph neural networks ====================

--- BM25 (Keywords) ---
1. [Score: 11.9965] graph neural networks for small graph and giant network representation learning an overview
   Link: https://arxiv.org/search/?query=%22graph%20neural%20networks%20for%20small%20graph%20and%20giant%20network%20representation%20learning%20an%20overview%22&searchtype=title
   Abstract: Graph neural networks denote a group of neural network models introduced for the representation lear...

2. [Score: 11.7769] a comprehensive survey on graph neural networks
   Link: https://arxiv.org/search/?query=%22a%20comprehensive%20survey%20on%20graph%20neural%20networks%22&searchtype=title
   Abstract: Deep learning has revolutionized many machine learning tasks in recent years, ranging from image cla...

3. [Score: 11.7012] graph neural processes towards bayesian graph neural networks
   Link: https://arxiv.org/search/?que

In [ ]:
from google.colab import files
from pypdf import PdfReader
from docx import Document
from IPython.display import display, Markdown
import os

def extract_text_from_file(path: str) -> str:
    """
    Extract plain text from a PDF, DOCX, or TXT file.
    """
    path = str(path)
    ext = os.path.splitext(path)[1].lower()

    if ext == ".pdf":
        text_chunks = []
        with open(path, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                try:
                    text_chunks.append(page.extract_text() or "")
                except Exception:
                    continue
        return "\n".join(text_chunks)

    elif ext in (".docx", ".doc"):
        doc = Document(path)
        return "\n".join(p.text for p in doc.paragraphs)

    elif ext in (".txt", ""):
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()

    else:
        raise ValueError(f"Unsupported file type: {ext}")


def summarize_text(text: str, max_chars=600) -> str:
    """
    Just for display: show a snippet of the uploaded doc.
    """
    text = text.strip()
    if len(text) <= max_chars:
        return text
    return text[:max_chars] + " ... [truncated]"


In [ ]:
import numpy as np

def recommend_from_document(engine,
                            top_k=10,
                            use_combined_if_available=True):
    """
    1. Upload a document (PDF/DOCX/TXT).
    2. Extract text.
    3. Run it through the search engine.
    4. Display recommended papers.
    """
    print("Please upload a PDF / DOCX / TXT file...")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded.")
        return

    # Take the first uploaded file
    filename = next(iter(uploaded.keys()))
    print(f"Received file: {filename}")

    # Save to local filesystem (Colab already does this, but we ensure path)
    path = filename

    # Extract text
    try:
        doc_text = extract_text_from_file(path)
    except Exception as e:
        print("Error extracting text:", e)
        return

    if not doc_text.strip():
        print("No text could be extracted from the document.")
        return

    display(Markdown("### Uploaded Document Snippet"))
    display(Markdown("```text\n" + summarize_text(doc_text) + "\n```"))

    # Decide which scorer to use
    use_combined = use_combined_if_available and "combined_scores" in globals()

    if use_combined:
        print("\nUsing combined BM25 + BERT + PageRank scorer...")
        results = combined_scores(engine, doc_text,
                                  w_bm25=0.3, w_bert=0.5, w_pr=0.2,
                                  top_k=top_k)
    else:
        print("\ncombined_scores not found – falling back to pure BERT.")
        results = engine._get_bert(doc_text, top_k=top_k)

    # Print recommendations
    display(Markdown("### Recommended Papers"))
    if not results:
        print("No recommendations found.")
        return

    for rank, (idx, score) in enumerate(results, start=1):
        row = engine.df.iloc[idx]
        title = row["title"]
        abstract = row["abstract"] if isinstance(row["abstract"], str) else ""
        link = engine._generate_link(title)

        md = f"""
**{rank}. {title}**
Score: `{score:.4f}`
[View on arXiv (title search)]({link})

> {abstract[:500]}{"..." if len(abstract) > 500 else ""}
"""
        display(Markdown(md))


In [ ]:
engine = PaperSearchEngine()


Loading search engine components...
System Ready!



In [ ]:
recommend_from_document(engine, top_k=10)


Please upload a PDF / DOCX / TXT file...


Saving 2307.07221v3 (1).pdf to 2307.07221v3 (1).pdf
Received file: 2307.07221v3 (1).pdf


### Uploaded Document Snippet

```text
1
Software Testing with Large Language Models:
Survey, Landscape, and Vision
Junjie Wang, Yuchao Huang, Chunyang Chen, Zhe Liu, Song Wang, Qing Wang
Abstract—Pre-trained large language models (LLMs) have recently emerged as a breakthrough technology in natural language
processing and artificial intelligence, with the ability to handle large-scale datasets and exhibit remarkable performance across a wide
range of tasks. Meanwhile, software testing is a crucial undertaking that serves as a cornerstone for ensuring the quality and reliability
of software products. As the scope and complexity of s ... [truncated]
```


combined_scores not found – falling back to pure BERT.


...


**6. on learning meaningful assert statements for unit test cases**  
Score: `0.6378`  
[View on arXiv (title search)](https://arxiv.org/search/?query=%22on%20learning%20meaningful%20assert%20statements%20for%20unit%20test%20cases%22&searchtype=title)

> Software testing is an essential part of the software lifecycle andrequires a substantial amount of time and effort. It has been esti-mated that software developers spend close to 50% of their time ontesting the code they write. For these reasons, a long standing goalwithin the research community is to (partially) automate softwaretesting. While several techniques and tools have been proposedto automatically generate test methods, recent work has criticizedthe quality and usefulness of the asser...



**7. about instruction sequence testing**  
Score: `0.6183`  
[View on arXiv (title search)](https://arxiv.org/search/?query=%22about%20instruction%20sequence%20testing%22&searchtype=title)

> Software testing is presented as a so-called theme within which different authors and groups have defined different subjects each of these subjects having a different focus on testing. A uniform concept of software testing is non-existent and the space of possible coherent perspectives on software testing, each fitting within the theme, is viewed as being spanned by five dimensions, each dimension representing two opposite views with a variety of intermediate views in between. Instruction sequen...



**8. metamorphic testing a new approach for generating next test cases**  
Score: `0.5984`  
[View on arXiv (title search)](https://arxiv.org/search/?query=%22metamorphic%20testing%20a%20new%20approach%20for%20generating%20next%20test%20cases%22&searchtype=title)

> In software testing, a set of test cases is constructed according to some predefined selection criteria. The software is then examined against these test cases. Three interesting observations have been made on the current artifacts of software testing. Firstly, an error-revealing test case is considered useful while a successful test case which does not reveal software errors is usually not further investigated. Whether these successful test cases still contain useful information for revealing s...



**9. active learning of input grammars**  
Score: `0.5957`  
[View on arXiv (title search)](https://arxiv.org/search/?query=%22active%20learning%20of%20input%20grammars%22&searchtype=title)

> Knowing the precise format of a program's input is a necessary prerequisite for systematic testing. Given a program and a small set of sample inputs, we (1) track the data flow of inputs to aggregate input fragments that share the same data flow through program execution into lexical and syntactic entities; (2) assign these entities names that are based on the associated variable and function identifiers; and (3) systematically generalize production rules by means of membership queries. As a res...



**10. an automated testing framework for conversational agents**  
Score: `0.5954`  
[View on arXiv (title search)](https://arxiv.org/search/?query=%22an%20automated%20testing%20framework%20for%20conversational%20agents%22&searchtype=title)

> Conversational agents are systems with a conversational interface that afford interaction in spoken language. These systems are becoming prevalent and are preferred in various contexts and for many users. Despite their increasing success, the automated testing infrastructure to support the effective and efficient development of such systems compared to traditional software systems is still limited. Automated testing framework for conversational systems can improve the quality of these systems by...


In [ ]:
# engine.py
import pickle
import torch
import pandas as pd
import networkx as nx
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import numpy as np
import urllib.parse
import os

# Optional: the permissive torch.load patch if you need it
_original_torch_load = torch.load
def permissive_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _original_torch_load(*args, **kwargs)
torch.load = permissive_load


class PaperSearchEngine:
    def __init__(self, data_dir="search_data"):
        self.data_dir = data_dir
        self.df = pd.read_pickle(os.path.join(data_dir, "df.pkl"))

        with open(os.path.join(data_dir, "graph.pkl"), "rb") as f:
            self.G = pickle.load(f)

        with open(os.path.join(data_dir, "bm25.pkl"), "rb") as f:
            self.bm25 = pickle.load(f)

        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.corpus_embeddings = torch.load(os.path.join(data_dir, "embeddings.pt"), weights_only=True)

    def _generate_link(self, title: str) -> str:
        safe_title = urllib.parse.quote(f'"{title}"')
        return f"https://arxiv.org/search/?query={safe_title}&searchtype=title"

    def _get_bm25(self, query: str, top_k: int = 10):
        toks = query.lower().split()
        scores = self.bm25.get_scores(toks)
        top_idx = np.argsort(scores)[::-1][:top_k]
        return [(int(i), float(scores[i])) for i in top_idx if scores[i] > 0]

    def _get_bert(self, query: str, top_k: int = 10):
        q_emb = self.model.encode(query, convert_to_tensor=True)
        cos_scores = util.cos_sim(q_emb, self.corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)
        return [(int(idx), float(score)) for idx, score in zip(top_results.indices, top_results.values)]

    def search_bert(self, query: str, top_k: int = 10):
        results = self._get_bert(query, top_k=top_k)
        return [self._format_result(idx, score) for idx, score in results]

    def search_bm25(self, query: str, top_k: int = 10):
        results = self._get_bm25(query, top_k=top_k)
        return [self._format_result(idx, score) for idx, score in results]

    def _format_result(self, idx: int, score: float):
        row = self.df.iloc[idx]
        return {
            "paper_id": int(row["paper_id"]),
            "title": row["title"],
            "abstract": row["abstract"],
            "score": score,
            "link": self._generate_link(row["title"]),
        }


In [ ]:
# api.py
from fastapi import FastAPI, UploadFile, File, HTTPException
from pydantic import BaseModel
from typing import List
import tempfile
import os

from engine import PaperSearchEngine
from pypdf import PdfReader
from docx import Document

app = FastAPI(title="Paper Search API")

# Load your engine once at startup
engine = PaperSearchEngine(data_dir="search_data")


# ---------- Models ----------

class SearchRequest(BaseModel):
    query: str
    top_k: int = 10
    mode: str = "bert"   # "bert" or "bm25"


class SearchResult(BaseModel):
    paper_id: int
    title: str
    abstract: str
    score: float
    link: str


# ---------- Helpers ----------

def extract_text_from_file(path: str) -> str:
    ext = os.path.splitext(path)[1].lower()

    if ext == ".pdf":
        text_chunks = []
        with open(path, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                text_chunks.append(page.extract_text() or "")
        return "\n".join(text_chunks)

    if ext in (".docx", ".doc"):
        doc = Document(path)
        return "\n".join(p.text for p in doc.paragraphs)

    if ext in (".txt", ""):
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()

    raise ValueError(f"Unsupported file type: {ext}")


# ---------- Endpoints ----------

@app.post("/search", response_model=List[SearchResult])
def search(req: SearchRequest):
    """
    Search by text query.
    - mode="bert": semantic search
    - mode="bm25": keyword search
    """
    if not req.query.strip():
        raise HTTPException(status_code=400, detail="Query must not be empty.")

    if req.mode == "bert":
        results = engine.search_bert(req.query, top_k=req.top_k)
    elif req.mode == "bm25":
        results = engine.search_bm25(req.query, top_k=req.top_k)
    else:
        raise HTTPException(status_code=400, detail="mode must be 'bert' or 'bm25'.")

    return results


@app.post("/recommend-from-doc", response_model=List[SearchResult])
async def recommend_from_doc(file: UploadFile = File(...), top_k: int = 10):
    """
    Upload a document (PDF/DOCX/TXT) and get recommended papers (BERT-based).
    """
    # Save to a temp file
    suffix = os.path.splitext(file.filename)[1]
    with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
        tmp.write(await file.read())
        tmp_path = tmp.name

    try:
        text = extract_text_from_file(tmp_path)
    except Exception as e:
        os.remove(tmp_path)
        raise HTTPException(status_code=400, detail=f"Error extracting text: {e}")

    os.remove(tmp_path)

    if not text.strip():
        raise HTTPException(status_code=400, detail="No text could be extracted from the document.")

    results = engine.search_bert(text, top_k=top_k)
    return results


@app.get("/health")
def health():
    return {"status": "ok"}


ModuleNotFoundError: No module named 'engine'

In [ ]:
# import os
# import random
# from tqdm import tqdm

# def prove_bert_superiority(engine, sample_size=200):
#     print(f"\n{'='*10} RUNNING 'TOPIC CONSISTENCY' EVALUATION {'='*10}")
#     print("Goal: Prove BERT understands the 'Field of Study' better than BM25/PageRank.")

#     # 1. Load Ground Truth Labels (The Subject Areas)
#     label_path = "dataset/ogbn_arxiv/raw/node-label.csv.gz"
#     if not os.path.exists(label_path):
#         print("Error: node-label.csv.gz not found. Cannot run evaluation.")
#         return

#     # Load labels (Shape: 169343)
#     node_labels = pd.read_csv(label_path, header=None).values.flatten()

#     # 2. Select Test Queries
#     # Ensure we only pick indices that exist in the dataframe
#     max_idx = min(len(engine.df), len(node_labels))
#     test_indices = random.sample(range(max_idx), sample_size)

#     methods = ['bm25', 'pagerank', 'bert']
#     scores = {m: [] for m in methods}

#     for idx in tqdm(test_indices):
#         # Use Abstract as query (Abstracts represent the "Topic" best)
#         query_text = engine.df.iloc[idx]['abstract']
#         true_label = node_labels[idx]

#         for method in methods:
#             # Get Top 10 Results
#             if method == 'bm25':
#                 results = engine._get_bm25(query_text, top_k=10)
#             elif method == 'bert':
#                 results = engine._get_bert(query_text, top_k=10)
#             elif method == 'pagerank':
#                 results = engine._get_pagerank(query_text, top_k=10)

#             # Calculate Precision: How many results share the same Subject Label?
#             matches = 0
#             count = 0
#             for res_idx, score in results:
#                 if res_idx < len(node_labels):
#                     if node_labels[res_idx] == true_label:
#                         matches += 1
#                     count += 1

#             if count > 0:
#                 scores[method].append(matches / count)

#     # 3. Print the "Winning" Chart
#     print(f"\n{'='*10} FINAL ACCURACY (Top-10) {'='*10}")
#     print(f"{'Method':<15} | {'Topic Accuracy':<15}")
#     print("-" * 35)

#     for method in methods:
#         accuracy = np.mean(scores[method]) * 100
#         print(f"{method.upper():<15} | {accuracy:.2f}%")

#     prove_bert_superiority(engine)